In [ ]:
import sys
sys.path.append('..')

In [ ]:
import pydeb
import numpy

In [ ]:
# Look up the Catalogue of Life (CoL) identifier for the species of interest
m=pydeb.infer.get_model_by_name('schol')
m.report()

In [ ]:
# Simulate entire life span (upto 10 x the age at which 99% of final length is reached)
# The age-maturity relationship from this simulation will be used as reference
delta_t= 0.01 * m.a_b
result = m.simulate(n=m.a_99/delta_t, delta_t=delta_t)
t_ref, E_H_ref = result['t'], result['E_H']

print('Age at birth: %.3f d vs simulated %.3f d' % (m.a_b, numpy.interp(m.E_Hb, E_H_ref, t_ref)))
if m.E_Hj > m.E_Hb:
    print('Age at metamorphosis: %.3f d vs simulated %.3f d' % (m.a_j, numpy.interp(m.E_Hj, E_H_ref, t_ref)))
print('Age at puberty: %.3f d vs simulated %.3f d' % (m.a_p, numpy.interp(m.E_Hp, E_H_ref, t_ref)))

precision = 0.001

import plotly.offline as py
import plotly.graph_objects as go

data = [go.Scatter(x=[m.a_b*0.001, m.a_p], y=[m.a_b*0.001, m.a_p], line={'color': 'black'}, mode='lines', name='reference (x=y)')]
E_H = numpy.logspace(numpy.log10(m.E_Hb*0.001), numpy.log10(m.E_Hb), 100)
t = [m.age_at_maturity(E_H, precision=precision) for E_H in E_H]
data.append(go.Scatter(x=numpy.interp(E_H, E_H_ref, t_ref), y=t, mode='markers', name='before birth'))
if m.E_Hj > m.E_Hb:
    E_H = numpy.logspace(numpy.log10(m.E_Hb), numpy.log10(m.E_Hj), 100)
    t = [m.age_at_maturity(E_H, precision=precision) for E_H in E_H]
    data.append(go.Scatter(x=numpy.interp(E_H, E_H_ref, t_ref), y=t, mode='markers', name='between birth and metamorphosis'))
E_H = numpy.logspace(numpy.log10(m.E_Hj), numpy.log10(m.E_Hp), 100)
t = [m.age_at_maturity(E_H, precision=precision) for E_H in E_H]
data.append(go.Scatter(x=numpy.interp(E_H, E_H_ref, t_ref), y=t, mode='markers', name='between metamorphosis (or birth) and puberty'))

layout = go.Layout(xaxis={'title': 'age from explicit simulation (d)', 'type': 'log'}, yaxis={'title': 'age from age_at_maturity (d)', 'type': 'log'})
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)